# 0. load package and split dataset

In [2]:
import numpy as np
import json
input_file = 'mnist_subset.json'
output_file = 'knn_output.txt'

with open(input_file) as json_data:
    data = json.load(json_data)

train_set, valid_set, test_set = data['train'], data['valid'], data['test']
Xtrain = train_set[0]
ytrain = train_set[1]
Xval = valid_set[0]
yval = valid_set[1]
Xtest = test_set[0]
ytest = test_set[1]

Xtrain = np.array(Xtrain)
Xval = np.array(Xval)
Xtest = np.array(Xtest)

ytrain = np.array(ytrain)
yval = np.array(yval)
ytest = np.array(ytest)

In [3]:
len(Xtrain)

5000

In [4]:
Xtrain.shape

(5000, 784)

# 1. compute distance matrix

In [67]:
###### Q5.1 ######
def compute_distances(Xtrain, X):
    """
    Compute the distance between each test point in X and each training point
    in Xtrain.
    Inputs:
    - Xtrain: A numpy array of shape (num_train, D) containing training data  
    - X: A numpy array of shape (num_test, D) containing test data.
    Returns:
    - dists: A numpy array of shape (num_test, num_train) where dists[i, j]
      is the Euclidean distance between the ith test point and the jth training
      point.
    """
    #####################################################
    #				 YOUR CODE HERE					                    #
    dists = np.sqrt(-2 * np.dot(X, Xtrain.T) + np.sum(Xtrain**2, axis = 1) + np.sum(X**2, axis = 1)[:, np.newaxis])
    #####################################################		 
    return dists

In [68]:
#==================Compute distance matrix=======================
dists = compute_distances(Xtrain, Xval)
dists.shape

(1000, 5000)

## Note
Vectorization trick to calculate distance - expand the L2 formula  
refer to https://medium.com/dataholiks-distillery/l2-distance-matrix-vectorization-trick-26aa3247ac6c 



# 2. Using distance matrix to predict 

In [81]:
def predict_labels(k, ytrain, dists):
    """
    Given a matrix of distances between test points and training points,
    predict a label for each test point.
    Inputs:
    - k: The number of nearest neighbors used for prediction.
    - ytrain: A numpy array of shape (num_train,) where ytrain[i] is the label
      of the ith training point.
    - dists: A numpy array of shape (num_test, num_train) where dists[i, j]
      gives the distance betwen the ith test point and the jth training point.
    Returns:
    - ypred: A numpy array of shape (num_test,) containing predicted labels for the
      test data, where y[i] is the predicted label for the test point X[i]. 
    """
    #####################################################
    #				 YOUR CODE HERE					                    #
    ypred=[]
    for i in range(len(dists)):
        idx=np.argpartition(dists[i,:],k)
        labelused=ytrain[idx[:k]]
        label,counts=np.unique(labelused,return_counts=True)
        label_counts=dict(zip(label,counts))
        label_pred=sorted(label_counts.items(),key=lambda x:(-x[1],x[0]))[0][0]
        ypred.append(label_pred)
    ypred=np.array(ypred)
    #####################################################
    return ypred

In [82]:
p=predict_labels(5,ytrain, dists)
p.shape

(1000,)

In [59]:
# Note: indexing for np.array
## find the index of max/min value
dists[4,:].argmin(axis=0)
## find the k smallest/largest values
idx=np.argpartition(dists[4,:])    # reorder the index from small to large
idx[:k]   # top k smallest values

TypeError: argpartition() missing 1 required positional argument: 'kth'

In [55]:
y=np.array([9,1,2,3,4,19,1,20])
np.argpartition(y,4)

array([1, 6, 2, 3, 4, 0, 5, 7])

# 3. compute prediction accuracy

In [72]:
###### Q5.3 ######
def compute_accuracy(y, ypred):
    """
    Compute the accuracy of prediction based on the true labels.
    Inputs:
    - y: A numpy array with of shape (num_test,) where y[i] is the true label
      of the ith test point.
    - ypred: A numpy array with of shape (num_test,) where ypred[i] is the 
      prediction of the ith test point.
    Returns:
    - acc: The accuracy of prediction (scalar).
    """
    #####################################################
    #				 YOUR CODE HERE					                    #
    acc=np.mean(y==ypred)
    #####################################################
    return acc

In [87]:
    k = 5
    ypred = predict_labels(k, ytrain, dists)
    acc = compute_accuracy(yval, ypred)
    print("The validation accuracy is", acc, "when k =", k)

The validation accuracy is 0.927 when k = 5


# 4. tune best K for KNN

In [74]:
###### Q5.4 ######
def find_best_k(K, ytrain, dists, yval):
    """
    Find best k according to validation accuracy.
    Inputs:
    - K: A list of ks.
    - ytrain: A numpy array of shape (num_train,) where ytrain[i] is the label
      of the ith training point.
    - dists: A numpy array of shape (num_test, num_train) where dists[i, j]
      is the Euclidean distance between the ith test point and the jth training
      point.
    - yval: A numpy array with of shape (num_val,) where y[i] is the true label
      of the ith validation point.
    Returns:
    - best_k: The k with the highest validation accuracy.
    - validation_accuracy: A list of accuracies of different ks in K.
    """

    #####################################################
    #				 YOUR CODE HERE					                    #
    validation_accuracy=[]
    for k in K:
        ypred = predict_labels(k, ytrain, dists)
        acc = compute_accuracy(yval, ypred)
        validation_accuracy.append(acc)
    best_k=K[validation_accuracy.index(max(validation_accuracy))]
    #####################################################
    return best_k, validation_accuracy

In [84]:
K=[1, 3, 5, 7, 9]
best_k,validation_accuracy = find_best_k(K, ytrain, dists, yval)
print(best_k)
print(validation_accuracy)

1
[0.94299999999999995, 0.93799999999999994, 0.93899999999999995, 0.92800000000000005, 0.92700000000000005]


In [85]:
dists = compute_distances(Xtrain, Xtest)
ypred = predict_labels(best_k, ytrain, dists)
test_accuracy = compute_accuracy(ytest, ypred)
print(test_accuracy)

0.932


## Overall

In [89]:
"""
Do not change the input and output format.
If our script cannot run your code or the format is improper, your code will not be graded.

The only functions you need to implement in this template is compute_distances, predict_labels, compute_accuracy
and find_best_k.
"""

import numpy as np
import json

###### Q5.1 ######
def compute_distances(Xtrain, X):
	"""
	Compute the distance between each test point in X and each training point
	in Xtrain.
	Inputs:
	- Xtrain: A numpy array of shape (num_train, D) containing training data
	- X: A numpy array of shape (num_test, D) containing test data.
	Returns:
	- dists: A numpy array of shape (num_test, num_train) where dists[i, j]
	  is the Euclidean distance between the ith test point and the jth training
	  point.
	"""
	#####################################################
	#				 YOUR CODE HERE					                    #
	dists = np.sqrt(-2 * np.dot(X, Xtrain.T) + np.sum(Xtrain**2, axis = 1) + np.sum(X**2, axis = 1)[:, np.newaxis])

	#####################################################		 
	return dists

###### Q5.2 ######
def predict_labels(k, ytrain, dists):
	"""
	Given a matrix of distances between test points and training points,
	predict a label for each test point.
	Inputs:
	- k: The number of nearest neighbors used for prediction.
	- ytrain: A numpy array of shape (num_train,) where ytrain[i] is the label
	  of the ith training point.
	- dists: A numpy array of shape (num_test, num_train) where dists[i, j]
	  gives the distance betwen the ith test point and the jth training point.
	Returns:
	- ypred: A numpy array of shape (num_test,) containing predicted labels for the
	  test data, where y[i] is the predicted label for the test point X[i]. 
	"""
	#####################################################
	#				 YOUR CODE HERE					                    #
	ypred=[]
	for i in range(len(dists)):
		idx=np.argpartition(dists[i,:],k)
		labelused=ytrain[idx[:k]]
		label,counts=np.unique(labelused,return_counts=True)
		label_counts=dict(zip(label,counts))
		label_pred=sorted(label_counts.items(),key=lambda x:(-x[1],x[0]))[0][0]
		ypred.append(label_pred)
	ypred=np.array(ypred)
	#####################################################
	return ypred

###### Q5.3 ######
def compute_accuracy(y, ypred):
	"""
	Compute the accuracy of prediction based on the true labels.
	Inputs:
	- y: A numpy array with of shape (num_test,) where y[i] is the true label
	  of the ith test point.
	- ypred: A numpy array with of shape (num_test,) where ypred[i] is the 
	  prediction of the ith test point.
	Returns:
	- acc: The accuracy of prediction (scalar).
	"""
	#####################################################
	#				 YOUR CODE HERE					                    #
	acc=np.mean(y==ypred)
	#####################################################
	return acc

###### Q5.4 ######
def find_best_k(K, ytrain, dists, yval):
	"""
	Find best k according to validation accuracy.
	Inputs:
	- K: A list of ks.
	- ytrain: A numpy array of shape (num_train,) where ytrain[i] is the label
	  of the ith training point.
	- dists: A numpy array of shape (num_test, num_train) where dists[i, j]
	  is the Euclidean distance between the ith test point and the jth training
	  point.
	- yval: A numpy array with of shape (num_val,) where y[i] is the true label
	  of the ith validation point.
	Returns:
	- best_k: The k with the highest validation accuracy.
	- validation_accuracy: A list of accuracies of different ks in K.
	"""
	
	#####################################################
	#				 YOUR CODE HERE					                    #
	validation_accuracy=[]
	for k in K:
		ypred = predict_labels(k, ytrain, dists)
		acc = compute_accuracy(yval, ypred)
		validation_accuracy.append(acc)
	best_k=K[validation_accuracy.index(max(validation_accuracy))]
	#####################################################
	return best_k, validation_accuracy


"""
NO MODIFICATIONS below this line.
You should only write your code in the above functions.
"""

def data_processing(data):
	train_set, valid_set, test_set = data['train'], data['valid'], data['test']
	Xtrain = train_set[0]
	ytrain = train_set[1]
	Xval = valid_set[0]
	yval = valid_set[1]
	Xtest = test_set[0]
	ytest = test_set[1]
	
	Xtrain = np.array(Xtrain)
	Xval = np.array(Xval)
	Xtest = np.array(Xtest)
	
	ytrain = np.array(ytrain)
	yval = np.array(yval)
	ytest = np.array(ytest)
	
	return Xtrain, ytrain, Xval, yval, Xtest, ytest
	
def main():
	input_file = 'mnist_subset.json'
	output_file = 'knn_output.txt'

	with open(input_file) as json_data:
		data = json.load(json_data)
	
	#==================Compute distance matrix=======================
	K=[1, 3, 5, 7, 9]	
	
	Xtrain, ytrain, Xval, yval, Xtest, ytest = data_processing(data)
	
	dists = compute_distances(Xtrain, Xval)
	
	#===============Compute validation accuracy when k=5=============
	k = 5
	ypred = predict_labels(k, ytrain, dists)
	acc = compute_accuracy(yval, ypred)
	print("The validation accuracy is", acc, "when k =", k)
	
	#==========select the best k by using validation set==============
	best_k,validation_accuracy = find_best_k(K, ytrain, dists, yval)

	
	#===============test the performance with your best k=============
	dists = compute_distances(Xtrain, Xtest)
	ypred = predict_labels(best_k, ytrain, dists)
	test_accuracy = compute_accuracy(ytest, ypred)
	
	#====================write your results to file===================
	f=open(output_file, 'w')
	for i in range(len(K)):
		f.write('%d %.3f' % (K[i], validation_accuracy[i])+'\n')
	f.write('%s %.3f' % ('test', test_accuracy))
	f.close()
	
if __name__ == "__main__":
	main()


The validation accuracy is 0.939 when k = 5
